# Preparing Data For Training
This notebook prepares the datasets created/preprocessed for training with `PointNet2`. **NOTE**: It is advised to have already preprocessed your data using the utility functions in `audi_dataset_pre_processing.ipynb` and `audi_dataset_pre_processing.py`. Please note that you may have to modify paths in the functions and scripts below.

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2 as cv
import json
import pickle
import pptk
from pyntcloud import PyntCloud
import copy
import operator
import warnings
warnings.filterwarnings('ignore')

## A2D2 DataLoader
This is the same DataLoader as found in the `data_utils/A2D2DataLoader.py` file.

In [ ]:
class A2D2DataLoader(Dataset):
    def __init__(self, dataset, rotation=None, \
                 normalize_xyz=True, normalize_rgb=True, \
                 take_subset=False, convert_to_tensor=True, \
                 target_ids=[3]):
                 #target_ids=[19, 54, 26, 46, 7, 18, 31, 24, 16, 53, 33,\
                             #37, 15, 13, 41, 3, 10, 22, 6, 42, 23, 14, \
                            # 5, 9, 12, 21, 48]):
        # Get IDS
        self.ids = list(dataset.keys())
        
        # Get rotation and length of dataset
        self.rotation = rotation
        self.N = len(self.ids)
        
        # Get geometric point cloud data and normalize
        self.xyz = [dataset[ID]['points'] for ID in self.ids]
        self.xyz_norm = self.normalize_xyz()
        
        # Get rgb data and normalize 
        self.rgb = [dataset[ID]['rgb'] for ID in self.ids]
        self.rgb_norm = self.normalize_rgb()
        
        # Combine xyz and rgb
        self.xyz_rgb = np.hstack((self.xyz, self.rgb))
        self.xyz_rgb_norm = [np.hstack((self.xyz_norm[i], self.rgb_norm[i])) for i in range(self.N)]
        
        # Get labels
        self.labels = [dataset[ID]['labels'] for ID in self.ids]
        
        # Get number of points to use
        self.num_points  = np.min([len(self.xyz[i]) for i in range(self.N)])
        print("SMALLEST PC POINTS: {}".format(self.num_points))
    
        if take_subset:
            self.target_ids = target_ids
            # Now get subset
            self.general_dataset, self.target_dataset = self.split_ds_by_classes()
        if convert_to_tensor:
            self.xyz_norm_tensor, self.rgb_norm_tensor, \
            self.xyz_rgb_norm_tensor, self.labels_tensor = self.convert_to_tensor()
    
    def __len__(self):
        return self.N

    def rotate_point_cloud_by_angle(self, data, rotation_angle):
        """
        Rotate the point cloud along up direction with certain angle.
        :param batch_data: Nx3 array, original batch of point clouds
        :param rotation_angle: range of rotation
        :return:  Nx3 array, rotated batch of point clouds
        """
        cosval = np.cos(rotation_angle)
        sinval = np.sin(rotation_angle)
        rotation_matrix = np.array([[cosval, 0, sinval],
                                    [0, 1, 0],
                                    [-sinval, 0, cosval]])
        rotated_data = np.dot(data, rotation_matrix)

        return rotated_data

    def __getitem__(self, index):
        if self.rotation is not None:
            index_xyz = self.xyz[index]
            angle = np.random.randint(self.rotation[0], self.rotation[1]) * np.pi / 180
            pointcloud = self.rotate_point_cloud_by_angle(index_xyz, angle)

            return pointcloud, self.labels[index]
        else:
            return self.xyz_rgb_norm_tensor[index], self.labels_tensor[index], len(self.xyz_rgb_norm_tensor[index])
    
    def normalize_xyz(self):
        normalized_xyz = []
        for ID in range(len(self.ids)):
            XYZ = np.copy(self.xyz[ID])
            centroid = np.mean(XYZ, axis=0)
            XYZ -= centroid
            furthest_distance = np.max(np.sqrt(np.sum(abs(XYZ)**2,axis=-1)))
            XYZ /= furthest_distance
            normalized_xyz.append(XYZ) 
        print("XYZ normalized")
        return normalized_xyz
    
    def normalize_rgb(self):
        normalized_rgb = []
        for ID in range(len(self.ids)):
            RGB = np.copy(self.rgb[ID])
            RGB = np.divide(RGB, 255.0)
            normalized_rgb.append(RGB)
        print("RGB normalized")
        return normalized_rgb
    
    def convert_to_tensor(self):
        """
        xyz_norm_tensor = torch.tensor(self.xyz_norm)
        rgb_norm_tensor = torch.tensor(self.rgb_norm)
        xyz_rgb_norm_tensor = torch.tensor(self.xyz_rgb_norm)
        labels_tensor = torch.tensor(self.labels)
        """
        xyz_norm_tensor = [torch.tensor(dp) for dp in self.xyz_norm]
        rgb_norm_tensor = [torch.tensor(dp) for dp in self.rgb_norm]
        xyz_rgb_norm_tensor = [torch.tensor(dp) for dp in self.xyz_rgb_norm]
        labels_tensor = [torch.tensor(dp) for dp in self.labels]
        
        return xyz_norm_tensor, rgb_norm_tensor, xyz_rgb_norm_tensor, labels_tensor
    
    def split_ds_by_classes(self):
        # Init output data structures
        gen_ds, target_ds = {}, {}
        
        general_id_indices = [j for j in range(55) if j not in self.target_ids]
        general_id_map = {general_id_indices[k]:k for k in range(len(general_id_indices))}
        
        # Now make subset and general ID maps and pickle them
        self.target_ids.sort()
        target_id_map = {self.target_ids[i]:i for i in range(len(self.target_ids))}
        target_id_map.update(general_id_map)
        
        print("Target ID Map: \n {}".format(target_id_map))
        print("General ID Map: \n {}".format(general_id_map))

        # Now pickle these
        f_out = os.path.join(os.getcwd(), "data", "ID_MAPS.pkl")
        with open(f_out, "wb") as f:
            pickle.dump([general_id_indices, general_id_map], f)
            f.close()
            
        for index in range(self.N): # Iterate over all images
            FOUND = False
            if index % 10000 == 0:
                print("Iterated through {} files".format(index))
            unique_ids = np.unique(self.labels[index])
            for ID in unique_ids:
                if ID in self.target_ids:
                    labels = self.labels[index]
                    mapped_labels = [target_id_map[labels[j]] for j in range(len(labels))]
                    target_ds[self.ids[index]] = {'points':self.xyz[index], 'labels':mapped_labels, 'rgb':self.rgb[index]}
                    FOUND = True
            if not FOUND:
                gen_ds[self.ids[index]] = {'points':self.xyz[index], 'labels':self.labels[index], 'rgb':self.rgb[index]}
        print("Number of pcs in general: {}, Number of pcs in target: {}".format(len(list(gen_ds.keys())),\
                                                                                 len(list(target_ds.keys()))))
        return gen_ds, target_ds

## DataLoader Wrapper Function

In [ ]:
def create_dataloader_wrapper(f_dataset, normalize_xyz=True, normalize_rgb=True,\
                              take_subset=False, convert_to_tensor=True):
    # Get input dataset
    with open(f_dataset, "rb") as f:
        dataset = pickle.load(f)
        f.close()

    # Instantiate the class object
    dataloader = A2D2DataLoader(dataset, normalize_xyz=normalize_xyz, normalize_rgb=normalize_rgb, \
                                take_subset=take_subset, convert_to_tensor=convert_to_tensor)
    return dataloader

## Loading Full Dataset with All Classes

In [ ]:
# Create datasets and save them

# Get input dataset
f_in = os.path.join(os.getcwd(), "data", "dataset_pc_labels_camera_start_0_stop_10000.pkl")
dataset = create_dataloader_wrapper(f_in, take_subset=False)
print("Finished processing dataset")

# Create output fname
f_out = os.path.join(os.getcwd(),"data","PROCESSED_mini_dataset_norm_tensor.pkl")

'''
# Pickle results
with open(f_out, "wb") as f:
    pickle.dump(dataset, f)
    f.close()
''' 
print("Pickled processed dataset to {}".format(f_out))

## Full Dataset with General and Target Classes

In [ ]:
# Get input dataset
f_in = os.path.join(os.getcwd(), "data", "dataset_pc_labels_camera_start_0_stop_10000_COMBINED_CLASSES.pkl")
dataset = create_dataloader_wrapper(f_in, take_subset=True)
print("Finished processing dataset")

In [ ]:
# Get general and target datasets
gen_ds, target_ds = dataset.general_dataset, dataset.target_dataset

### Process General Dataset

In [ ]:
# Now create A2D2Dataloader Objects based off of these datasets
gen_dataset = A2D2DataLoader(gen_ds, normalize_xyz=True, normalize_rgb=True, \
                                take_subset=False, convert_to_tensor=True)

print("UNIQUE LABELS: {}".format(np.unique(gen_dataset.labels[0])))
      
# Create output fname
f_out_general = os.path.join(os.getcwd(),"data","PROCESSED_general_dataset_start_0_stop_10000_COMBINED CLASSES.pkl")


# Pickle results - general
with open(f_out_general, "wb") as f:
    pickle.dump(gen_ds, f)
    f.close()

print("Pickled general processed dataset to {}".format(f_out_general))


### Process Target Dataset

In [ ]:
target_dataset = A2D2DataLoader(target_ds, normalize_xyz=True, normalize_rgb=True, \
                                take_subset=False, convert_to_tensor=True)

print("UNIQUE LABELS: {}".format(np.unique(target_dataset.labels[0])))

# Create output fname
f_out_target = os.path.join(os.getcwd(),"data","PROCESSED_target_dataset_start_0_stop_10000_COMBINED_CLASSES.pkl")

# Pickle results - general
with open(f_out_target, "wb") as f:
    pickle.dump(gen_ds, f)
    f.close()

print("Pickled target processed dataset to {}".format(f_out_target))

## Plot Histogram of Points over Point Clouds

In [ ]:
# Get distribution of number of training points
lengths = []
for i in range(dataset.N):
    lengths.append(min(len(dataset.xyz[i]), 20000))

# Now plot
plt.hist(lengths,50)
plt.title("Histogram of Number of Points in Point Cloud for A2D2 Dataset")
plt.show()

## Sanity Check for Dataset Loading and Pre-processing

In [ ]:
def test_load_datasets(f_pickle):
    with open(f_pickle, "rb") as f:
        D = pickle.load(f)
        f.close()
    print(D.xyz_norm_tensor[0].numpy().shape)
    print(D.rgb_norm_tensor[0].numpy().shape)
    print(D.xyz_rgb_norm_tensor[0].numpy().shape)
    print(D.labels_tensor[0].numpy().shape)
    
    print("MAX XYZ", np.max(D.xyz_norm_tensor[0].numpy()))
    print("MIN XYZ", np.min(D.xyz_norm_tensor[0].numpy()))
    print("MAX RGB", np.max(D.rgb_norm_tensor[0].numpy()))
    print("MIN RGB", np.min(D.rgb_norm_tensor[0].numpy()))
    print("MAX LABEL", np.max(D.labels_tensor[0].numpy()))
    print("MIN LABEL", np.min(D.labels_tensor[0].numpy()))

    print("GET MIN NUMBER OF POINTS")
    

# Run test
test_load_datasets(
    os.path.join(os.getcwd(),"data","PROCESSED_mini_dataset_norm_tensor.pkl"))

## Use to find which labels are present in which classes

In [ ]:
def create_seen_matrix(dataloader):
    # Now we want to determine which classes appear the fewest number of times
    seen = {i: 0 for i in range(55)}
    print(seen)
    count = 0
    for index in range(dataloader.N): # Iterate over each image
        if count % 1000 == 0:
            print("Iterated through {} point clouds".format(count))
        seen_i = {j: 0 for j in range(55)}
        for label in dataloader.labels[index]:
            seen_i[label] = 1
        copy_seen = copy.deepcopy(seen)
        seen = {k:seen_i[k]+copy_seen[k] for k in range(55)}
        count += 1
    return seen

## Find the rarest classes for transfer learning

In [ ]:
def get_n_rarest_classes(dataloader,n=10): 
    seen = create_seen_matrix(dataloader)
    sorted_seen = sorted(seen.items(), key=operator.itemgetter(1))
    return sorted_seen

In [ ]:
# Run call for function above
get_n_rarest_classes(dataset, n=55)

## Get Label Counts Across the Entire Dataset
The code block below can be used to extract the label counts for datasets with a different number of semantic classes.

In [ ]:
# Class imbalance affects performance - let's fix that!
full_dataset_file = os.path.join(os.getcwd(), "data", "dataset_pc_labels_camera_start_0_stop_28652.pkl")

# Also compare for combined classes dataset
combined_classes_dataset = os.path.join(os.getcwd(), "data", \
                            "dataset_pc_labels_camera_start_0_stop_10000_COMBINED_CLASSES.pkl")

# Also compare for road detection dataset
road_detection_dataset = os.path.join(os.getcwd(), "data", \
                            "dataset_pc_labels_camera_start_0_stop_10000_ROAD_DETECTION.pkl")

# Import pickle file
with open(road_detection_dataset, "rb") as f:
    D = pickle.load(f)
    f.close()

    
full_label_counts = {i:0 for i in range(55)}
combined_label_counts = {i:0 for i in range(6)}
road_detection_label_counts = {i:0 for i in range(2)}
    
keys = list(D.keys())
count = 0

for key in keys:
    if count % 1000 == 0:
        print("Processed {} files".format(count))
    for label in D[key]['labels']:
        road_detection_label_counts[label] += 1
    count += 1

print(road_detection_label_counts)

## Pickle Class Weights File

In [ ]:
# Pickle this dictionary to file
class_weights_fpath = os.path.join(os.getcwd(), "data", "class_weights_ROAD_DETECTION.pkl")

print(road_detection_label_counts)

with open(class_weights_fpath, "wb") as f:
    pickle.dump(road_detection_label_counts, f)
    f.close()